In [1]:
# import package

# model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import timm

# dataset and transformation
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import os

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import os
import numpy as np
import time
import copy
import random

import PIL
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  2.5.1+cu118
Torchvision Version:  0.20.1+cu118


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()
device

device(type='cuda')

In [3]:
class_mapping = {
            '27_30': '0',
            '27_45': '1',
            # '28': '2',
            # '31': '3'
        }

class LabelingDataset(Dataset):
    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)

        img_transformed = self.transform(img, self.phase)

        label = img_path.split('/')[-2].split('.')[0]
        label = int(class_mapping[label])

        return img_transformed, label


class ImageTransform():
    def __init__(self, resize):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.Resize((resize, resize)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor()
            ]),
            'val': transforms.Compose([
                transforms.Resize((resize, resize)),
                transforms.ToTensor()
            ])
        }

    def __call__(self, img, phase):
        img_rgb = img.convert('RGB')
        tensor_image = self.data_transform[phase](img_rgb)
        return tensor_image
        
transform = ImageTransform(299)

In [4]:
BASE_DIR = '/home/dlit_ai/seohan_data/origin'
image_file_list = []
for class_name in class_mapping:
    images_filepaths = sorted([os.path.join(BASE_DIR, class_name, f) for f in os.listdir(os.path.join(BASE_DIR, class_name))])
    print(len(images_filepaths))
    image_file_list.append(images_filepaths)

33431
13255


In [5]:
#원래코드
min_class_count = min(len(image_list) for image_list in image_file_list)  # 가장 작은 클래스의 데이터 개수 확인
val_count = min_class_count // 10

train_images_filepaths =[]
val_images_filepaths = []

for image_list in image_file_list:
    correct_image_filepaths = [filepath for filepath in image_list if '.bmp' in filepath]
    random.seed(77)
    random.shuffle(correct_image_filepaths)
    correct_image_filepaths = correct_image_filepaths[:min_class_count]
    train = correct_image_filepaths[:-val_count]
    test = correct_image_filepaths[-val_count:]
    
    train_images_filepaths += train
    val_images_filepaths += test

print(len(train_images_filepaths), len(val_images_filepaths))

23860 2650


In [6]:
size = 299
batch_size = 32

# batch_size = 8

num_classes = 2
num_epochs = 10
model_name = 'inception_resnet_v2'

In [7]:
train_dataset = LabelingDataset(train_images_filepaths, transform=ImageTransform(size), phase='train')
val_dataset = LabelingDataset(val_images_filepaths, transform=ImageTransform(size), phase='val')

train_iterator  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_iterator = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

dataloader_dict = {'train': train_iterator, 'val': valid_iterator}

In [8]:
model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)
# model.load_state_dict(torch.load('./inception_v4_final_ll_6.pth'))
# checkpoint = torch.load('./inception_v4_0006_7.pth')
# model.load_state_dict(checkpoint['model_state_dict'])

model = model.to(device)

In [9]:
def train_model(model, dataloaders, criterion, optimizer, model_path, num_epochs=25, epoch_st=0, epoch_acc=0.0):
    since = time.time()
    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = epoch_acc

    for epoch in range(epoch_st, num_epochs):
        epoch_start = time.time()

        print('Epoch {}/{}, Start Time:{}'.format(epoch, num_epochs-1, epoch_start))
        print('-' * 10)

        # 에폭 하나를 수행할 때마다 training과 validation을 수행
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() #학습모드
            else:
                model.eval() #평가모드

            running_loss = 0.0 # loss 초기화
            running_corrects = 0 # corrects 초기화

            batch_idx = 0
            for inputs, labels in dataloaders[phase]:
                print(batch_idx, inputs.size())

                # print(f"Device in use: {inputs.device}, Model on: {next(model.parameters()).device}")

                batch_idx += 1
                inputs = inputs.to(device)
                labels = labels.to(device)

                # print(f"After to(device): Inputs on: {inputs.device}, Labels on: {labels.device}")


                #매개변수 경사도를 0으로 설정
                optimizer.zero_grad()

                # 순전파
                # 학습 시에만 연산 기록을 추적
                # print(phase)
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)
                    #print(labels, preds)

                    # 학습 단계인 경우 역전파 + 최적화
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data) # prediction이 바르게 횟수
                # print("running done")

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            # double() 메서는 tensor의 내장 메서드 tensor element를 모두 double 형으로 변환

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            epoch_time_elapsed = time.time() - epoch_start
            print('{:.0f} Epoch Training complete in {:.0f}m {:.0f}s'.format(epoch, epoch_time_elapsed // 60, epoch_time_elapsed % 60))
            # 모델을 깊은 복사(deep copy)함
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

            # epoch 끝날 때 마다 모델 저장
            PATH =  model_path + f'{model_name}_2class_2_{epoch:04d}.pth'
            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss,
            'accuracy': epoch_acc,
            }, PATH)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss,
            'accuracy': epoch_acc,
            }, model_path + model_name +'_2class_2_final.pth')

    return model, val_acc_history


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model.parameters(), lr=0.001)

In [11]:
model_path = './new_model/'
model_ft, hist = train_model(model, dataloader_dict, criterion, optimizer_ft, model_path, num_epochs=num_epochs)

torch.save(model_ft.state_dict(), model_path + model_name +'_2class_2_final_ll.pth')

Epoch 0/9, Start Time:1736822435.5669765
----------
0 torch.Size([32, 3, 299, 299])
1 torch.Size([32, 3, 299, 299])
2 torch.Size([32, 3, 299, 299])
3 torch.Size([32, 3, 299, 299])
4 torch.Size([32, 3, 299, 299])
5 torch.Size([32, 3, 299, 299])
6 torch.Size([32, 3, 299, 299])
7 torch.Size([32, 3, 299, 299])
8 torch.Size([32, 3, 299, 299])
9 torch.Size([32, 3, 299, 299])
10 torch.Size([32, 3, 299, 299])
11 torch.Size([32, 3, 299, 299])
12 torch.Size([32, 3, 299, 299])
13 torch.Size([32, 3, 299, 299])
14 torch.Size([32, 3, 299, 299])
15 torch.Size([32, 3, 299, 299])
16 torch.Size([32, 3, 299, 299])
17 torch.Size([32, 3, 299, 299])
18 torch.Size([32, 3, 299, 299])
19 torch.Size([32, 3, 299, 299])
20 torch.Size([32, 3, 299, 299])
21 torch.Size([32, 3, 299, 299])
22 torch.Size([32, 3, 299, 299])
23 torch.Size([32, 3, 299, 299])
24 torch.Size([32, 3, 299, 299])
25 torch.Size([32, 3, 299, 299])
26 torch.Size([32, 3, 299, 299])
27 torch.Size([32, 3, 299, 299])
28 torch.Size([32, 3, 299, 299])
2

In [12]:
hist

[tensor(0.8389, device='cuda:0', dtype=torch.float64),
 tensor(0.9894, device='cuda:0', dtype=torch.float64),
 tensor(0.9992, device='cuda:0', dtype=torch.float64),
 tensor(0.9992, device='cuda:0', dtype=torch.float64),
 tensor(0.9792, device='cuda:0', dtype=torch.float64),
 tensor(0.9958, device='cuda:0', dtype=torch.float64),
 tensor(0.9996, device='cuda:0', dtype=torch.float64),
 tensor(0.9985, device='cuda:0', dtype=torch.float64),
 tensor(0.9992, device='cuda:0', dtype=torch.float64),
 tensor(0.5098, device='cuda:0', dtype=torch.float64)]